# https://github.com/mturk24/bandits
# https://www.analyticsvidhya.com/blog/2018/09/reinforcement-multi-armed-bandit-scratch-python/#comment-155315

In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
# Importing the dataset

dataset = pd.read_csv('/Users/mingzhangyin/Downloads/bandits/Ads_Optimisation.csv')

In [12]:
# pandas df way to get first 5
print(dataset.shape)
dataset.head()

(10000, 10)


Ad 1  Ad 2  Ad 3  Ad 4  Ad 5  Ad 6  Ad 7  Ad 8  Ad 9  Ad 10
0     1     0     0     0     1     0     0     0     1      0
1     0     0     0     0     0     0     0     0     1      0
2     0     0     0     0     0     0     0     0     0      0
3     0     1     0     0     0     0     0     1     0      0
4     0     0     0     0     0     0     0     0     0      0

In [4]:
# pandas df way to get last 5
dataset.tail()

Ad 1  Ad 2  Ad 3  Ad 4  Ad 5  Ad 6  Ad 7  Ad 8  Ad 9  Ad 10
9995     0     0     1     0     0     0     0     1     0      0
9996     0     0     0     0     0     0     0     0     0      0
9997     0     0     0     0     0     0     0     0     0      0
9998     1     0     0     0     0     0     0     1     0      0
9999     0     1     0     0     0     0     0     0     0      0

In [5]:
# Implementing Random Selection
def random_selection():
    N = 10000
    d = 10
    ads_selected = []
    total_reward = 0
    for n in range(0, N):
        ad = random.randrange(d)
        ads_selected.append(ad)
        # chooses the value at index ad in the nth row of the dataset where there are 10000 rows (0 - 9999 index in 2d array)
        reward = dataset.values[n, ad]
        total_reward = total_reward + reward
    return total_reward



In [6]:
# Observe reward of choosing ads randomly and never exploiting the optimal one
random_result = random_selection()
print(random_result)

1190


In [7]:
# Test out 100 sample runs of the random selection algorithm and average the result
total_reward = 0
for i in range(1, 101):
    random_result_i = random_selection()
    total_reward += random_result_i
avg_reward = total_reward/100
print(avg_reward)  

1233.33


In [8]:
# Epsilon Greedy Approach
def epsilon_greedy(epsilon):
    N = 10000
    d = 10
    ads_selected = []
    # keep track of how many times actions are taken
    numbers_of_selections = [0] * d
    sums_of_reward = [0] * d
    # keep track of total reward (q function)
    total_reward = 0
    rand = np.random.random_integers(0, 1)   
    for n in range(0, N):
        ad = 0
        max_q_value = 0
        for i in range(0, d):
            if (numbers_of_selections[i] > 0):
                if rand <= epsilon:
                    ad = random.randrange(d)
                else:
                    # calculate q(a) or avg reward of all rewards for choosing ad i divided by number of times choosing ad i
                    q_val = sums_of_reward[i] / numbers_of_selections[i]
                    if q_val > max_q_value:
                        max_q_value = q_val
                        ad = i   
        ads_selected.append(ad)
        numbers_of_selections[ad] += 1
        reward = dataset.values[n, ad]
        sums_of_reward[ad] += reward
        total_reward += reward
    return total_reward
    



In [9]:
# Observe reward of choosing ads according to an epsilon-greedy policy
epsilon_result = epsilon_greedy(0.5)
print(epsilon_result)

1703


/var/folders/b6/dnmgz4v90qndj_wtz10vd5rm0000gn/T/ipykernel_42285/928509845.py:11: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  rand = np.random.random_integers(0, 1)


In [10]:
# Implementing UCB
def upper_confidence_bound():
    N = 10000
    d = 10
    ads_selected = []
    # keep track of how many times actions are taken
    numbers_of_selections = [0] * d
    sums_of_reward = [0] * d
    # keep track of total reward (q function)
    total_reward = 0
    for n in range(0, N):
        ad = 0
        max_upper_bound = 0
        for i in range(0, d):
            if (numbers_of_selections[i] > 0):
                # calculate q(a) or avg reward of all rewards for choosing ad i divided by number of times choosing ad i
                average_reward = sums_of_reward[i] / numbers_of_selections[i]

                # calculate the confidence of the upper bound
                delta_i = math.sqrt(2 * math.log(n+1) / numbers_of_selections[i])

                # total upper bound is q(a) + confidence
                upper_bound = average_reward + delta_i     
            else:
                upper_bound = 1000
            if upper_bound > max_upper_bound:
                max_upper_bound = upper_bound
                ad = i
        ads_selected.append(ad)
        numbers_of_selections[ad] += 1
        reward = dataset.values[n, ad]
        sums_of_reward[ad] += reward
        total_reward += reward
    return total_reward
    

In [11]:
# Observe reward of choosing ads according to an upper confidence bound policy
ucb_result = upper_confidence_bound()
print(ucb_result)


2125
